In [2]:
!pip install pytorch-nlp
from torchnlp.datasets import imdb_dataset
from tqdm import tqdm
import re
from nltk import tokenize
import nltk
import pickle
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
################################################################################
drive.mount('/content/drive', force_remount=True)
nltk.download('punkt')
train = imdb_dataset(train=True)
test = imdb_dataset(test=True)
################################################################################
def tokenizing(data, mode=0):
    if mode==1:
        text = ' '.join([s["text"] for s in data])
    else:
        text = data
    text = text.replace('<br /><br />',' ')
    text = text.replace('\x85','')
    text = text.lower()
    tokenized_sentence = re.compile('[.!?()]').split(text)
    tokenized_corpus = [None]*len(tokenized_sentence)
    if mode==1:
        for i in tqdm(range(len(tokenized_sentence))):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    else:
        for i in range(len(tokenized_sentence)):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    return tokenized_corpus
################################################################################
class MyDataset(Dataset):
  def __init__(self, x, y, z):
    self.data = x
    self.labels = y
    self.lengths = z

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index], self.lengths[index]
################################################################################
def custom_collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    _, labels, lengths = zip(*data)
    max_len = max(lengths)
    fix_size = data[0][0].size(1)
    features = torch.zeros((len(data), 1, fix_size, max_len))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
      k, j = data[i][0][0].size(0), data[i][0][0].size(1)
      features[i][0] = torch.cat((data[i][0][0], torch.zeros((k, max_len - j))),1)

    return features.float(), labels.long(), lengths.long()
################################################################################
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, output, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        # _, predicted = torch.max(output.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        # self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total
################################################################################
def vec(word):
  try:
    a = model.wv[word]
  except:
    a = np.zeros(300)
  return a

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model = pickle.load(open('/content/drive/MyDrive/NLP/w2v_model_gensim.p','rb'))

In [3]:
model = pickle.load(open('/content/drive/MyDrive/NLP/glove-wiki-gigaword-300.p','rb'))

In [ ]:
random.shuffle(train)
vector_size = 300
x = []
y = []
z = []
for a in tqdm(train):
  doc = tokenize.word_tokenize(a['text'])
  x.append(torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))))
  y.append([int(a['sentiment']=='pos')])
  z.append(len(doc))
train_dataset = MyDataset(x,y,z)

random.shuffle(test)
x = []
y = []
z = []
for a in tqdm(test[0:1000]):
  doc = tokenize.word_tokenize(a['text'])
  x.append(torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))))
  y.append([int(a['sentiment']=='pos')])
  z.append(len(doc))
val_dataset = MyDataset(x,y,z)

 63%|██████▎   | 15636/25000 [00:57<00:41, 227.17it/s]

In [ ]:
random.shuffle(train)
vector_size = 300
x = []
y = []
z = []
for a in tqdm(train):
  doc = tokenize.word_tokenize(a['text'])
  x.append(torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))))
  y.append([int(a['sentiment']=='pos')])
  z.append(len(doc))
train_dataset = MyDataset(x,y,z)

random.shuffle(test)
x = []
y = []
z = []
for a in tqdm(test[0:1000]):
  doc = tokenize.word_tokenize(a['text'])
  x.append(torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))))
  y.append([int(a['sentiment']=='pos')])
  z.append(len(doc))
val_dataset = MyDataset(x,y,z)

 65%|██████▍   | 16242/25000 [00:53<00:26, 327.10it/s]

In [9]:
a =  torch.zeros(2,2,2)
a[0]

tensor([[0., 0.],
        [0., 0.]])

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, collate_fn=custom_collate_fn, batch_size=50, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, collate_fn=custom_collate_fn, batch_size=50, shuffle=True, num_workers=2)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,2))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,3))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,4))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,5))
        self.fc1 = nn.Linear(800, 1)
        self.dropout = nn.Dropout(0.50)
    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x1 = F.max_pool2d(x1, (1,x1.size()[-1]))
        x2 = F.relu(self.conv2(x))
        x2 = F.max_pool2d(x2, (1,x2.size()[-1]))
        x3 = F.relu(self.conv3(x))
        x3 = F.max_pool2d(x3, (1,x3.size()[-1]))
        x4 = F.relu(self.conv4(x))
        x4 = F.max_pool2d(x4, (1,x4.size()[-1]))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),1),start_dim=1)
        x = self.fc1(self.dropout(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.BCEWithLogitsLoss()

In [8]:
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.001)
net = net.float()
net.train()
accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs.float())
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels.float())
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * accuracy.compute()))

print('Finished Training')


Starting epoch 1


loss: 0.01425|acc: 49.42%: 100%|██████████| 100/100 [00:14<00:00,  6.96it/s]


Starting epoch 2



loss: 0.01248|acc: 59.82%: 100%|██████████| 100/100 [00:15<00:00,  6.65it/s]


Starting epoch 3


...


Starting epoch 14



loss: 0.00321|acc: 93.48%: 100%|██████████| 100/100 [00:14<00:00,  6.84it/s]


Starting epoch 15



loss: 0.00294|acc: 94.70%:  43%|████▎     | 43/100 [00:07<00:10,  5.48it/s]


KeyboardInterrupt: ignored

In [ ]:
pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
net = pickle.load(open('/content/drive/MyDrive/NLP/imdb_sentiment.p','rb'))

In [12]:
net.eval()
accuracy = Accuracy()
accuracy.reset()
# Gradients are calculated on the forward pass for every iteration.
# As we do not need gradients now, we can disable the calculation.
with torch.no_grad():
    for data in tqdm(train_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)

print("Training Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 100/100 [00:13<00:00,  7.34it/s]

Training Accuracy: 100.00%


In [11]:
net.eval()
accuracy = Accuracy()
accuracy.reset()        
with torch.no_grad():
    for data in tqdm(val_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)
        
print("\nTesting Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 20/20 [00:02<00:00,  8.05it/s]


Testing Accuracy: 53.00%
